## RAG: Retrieval Augmented Generation.
- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query -> Search -> Results -> (LLM) -> Answer

In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pdfs/kim-tse-2008.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [5]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [12]:
chain.invoke({"question": "What is bug classficiation?", "Context": docs})

'What is bug classification?\n\nBug classification is a new technique for predicting latent software bugs, called change classification. Change classification uses a machine learning classifier to determine whether a new software change is more similar to prior buggy changes or clean changes. In this manner, change classification predicts the existence of bugs in software changes. The classifier is trained using features (in the machine learning sense) extracted from the revision history of a software project stored in its software configuration management repository. The trained classifier can classify changes as buggy or clean, with a 78 percent accuracy on average and a 60 percent buggy change recall on average. Change classification has several desirable qualities: 1) The prediction granularity is small (a change to a single file), 2) predictions do not require semantic information about the source code, 3) the technique works for a broad array of project types and programming lang

In [8]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)

retriever = BM25Retriever.from_documents(splits)

In [9]:
retriever.invoke("What is bug classficiation?")

[Document(page_content="<p id='102' style='font-size:16px'>One assumption of the presentation so far is that a bug is<br>repaired in a single bug-fix change. What happens when a<br>bug is repaired across multiple commits? There are two<br>cases. In the first case, a bug repair is split across multiple<br>commits, with each commit modifying a separate section of<br>the code (code sections are disjoint). Each separate change is<br>tracked back to its initial bug-introducing change, which is<br>then used to train the SVM classifier. In the second case, a bug<br>fix occurs incrementally over multiple commits, with some<br>later fixes modifying earlier ones (the fix code partially<br>overlaps). The first patch in an overlapping code section<br>would be traced back to the original bug-introducing change.<br>Later modifications would not be traced back to the original<br>bug-introducing change. Instead, they would be traced back<br>to an intermediate modification, which is identified as bug",

In [10]:
query = "What is bug classficiation?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'The information is not present in the context.'

In [11]:
query = "What is bug classficiation?"
context_docs = retriever.invoke("bug")
chain.invoke({"question": query, "Context": context_docs})

'Bug classification is a technique that predicts whether there is a bug in any of the lines that were changed in one file in one SCM commit transaction. It differs from previous bug prediction work that focuses on identifying fault-prone or buggy modules, files, and functions. Bug classification allows for immediate bug predictions as soon as a change is made.'

# Excercise 
It seems keyword search is not the best for LLM queries. What are some alternatives?